In [1]:
%cd /home/ubuntu/environment/SY/assets
%pip install -r requirements.txt

/home/ubuntu/environment/SY/assets


/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
ls

consolidate_adapter_shards_and_merge_model.py  finetune_model.py
finetune_emergency_response.py                 requirements.txt


# Fine-tuning

In this section, we fine-tune the Qwen3-1.7B model on the Text-to-SQL task using Hugging Face Optimum Neuron. Here are the parameters we are going to pass - 

1. `--nnodes`:	Number of nodes (1 = single node)
2. `--nproc_per_node`: 	Processes per node (usually equals number of devices).
3. `--model_id, --tokenizer_id`:	Model and tokenizer identifiers (from Hugging Face or local path).
4. `--output_dir`:	Directory for saving checkpoints and logs.
5. `--bf16`:	Enables bfloat16 precision for faster, memory-efficient training.
5. `--gradient_checkpointing`:	Saves memory by recomputing activations during backprop.
6. `--gradient_accumulation_steps`:	Steps to accumulate gradients before optimizer update.
7. `--learning_rate`:	Initial training learning rate.
8. `--max_steps`:	Total number of training steps.
9. `--per_device_train_batch_size`:	Batch size per device.
10. `--tensor_parallel_size`:	Number of devices for tensor parallelism.
11. `--lora_r, --lora_alpha, --lora_dropout`:	LoRA hyperparameters — rank, scaling, and dropout rate.
12. `--dataloader_drop_last`:	Drops last incomplete batch.
13. `--disable_tqdm`: Disables progress bar.
14. `--logging_steps`:	Log interval (in steps).

In [3]:
!torchrun \
  --nnodes 1 \
  --nproc_per_node 2 \
  finetune_emergency_response.py \
  --model_id Qwen/Qwen3-1.7B \
  --tokenizer_id Qwen/Qwen3-1.7B \
  --output_dir ~/environment/ml/SY/qwen \
  --bf16 True \
  --gradient_checkpointing True \
  --gradient_accumulation_steps 1 \
  --learning_rate 5e-5 \
  --max_steps 3500 \
  --per_device_train_batch_size 2 \
  --tensor_parallel_size 2 \
  --lora_r 16 \
  --lora_alpha 32 \
  --lora_dropout 0.05 \
  --dataloader_drop_last True \
  --disable_tqdm True \
  --logging_steps 10

W1108 23:16:05.432000 195526 /opt/aws_neuronx_venv_pytorch_2_7_nxd_inference/lib/python3.10/site-packages/torch/distributed/run.py:766] 
W1108 23:16:05.432000 195526 /opt/aws_neuronx_venv_pytorch_2_7_nxd_inference/lib/python3.10/site-packages/torch/distributed/run.py:766] *****************************************
W1108 23:16:05.432000 195526 /opt/aws_neuronx_venv_pytorch_2_7_nxd_inference/lib/python3.10/site-packages/torch/distributed/run.py:766] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1108 23:16:05.432000 195526 /opt/aws_neuronx_venv_pytorch_2_7_nxd_inference/lib/python3.10/site-packages/torch/distributed/run.py:766] *****************************************
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is 

# Compilation

After completing the fine-tuning process, the next step is to compile the trained model for AWS Trainium inference using the Hugging Face Optimum Neuron toolchain.
Neuron compilation optimizes the model graph and converts it into a Neuron Executable File Format (NEFF), enabling efficient execution on NeuronCores.

In [4]:
!optimum-cli export neuron \
  --model /home/ubuntu/environment/ml/SY/qwen/merged_model \
  --task text-generation \
  --sequence_length 512 \
  --batch_size 1 \
  /home/ubuntu/environment/ml/SY/qwen/compiled_model

/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/optimum/commands/env.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution


/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/modules/moe/blockwise.py:42: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  component, error = import_nki(config)
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/modules/moe/blockwise

# Inference

We will install the Optimum Neuron vllm library.  Then, run inference using the compiled model.

In [5]:
%pip install optimum-neuron[vllm]


Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
from vllm import LLM, SamplingParams
llm = LLM(
    model="/home/ubuntu/environment/ml/SY/qwen/compiled_model", #local compiled model
    max_num_seqs=1,
    max_model_len=2048,
    device="neuron",
    tensor_parallel_size=2,
    override_neuron_config={})

prompts = [
"""
<|im_start|>system
You are a calm, medically accurate first-aid assistant. Give step-by-step help.<|im_end|>
<|im_start|>user
My friend burned his hand on a hot pan. What should I do immediately?<|im_end|>
<|im_start|>assistant
""",
"""
<|im_start|>system
You are an offline emergency chatbot providing safe, concise first-aid advice.<|im_end|>
<|im_start|>user
Someone nearby fainted suddenly, what’s the first thing I should check?<|im_end|>
<|im_start|>assistant
"""
]

sampling_params = SamplingParams(max_tokens=2048, temperature=0.75)
outputs = llm.generate(prompts, sampling_params)

print("#########################################################")

for output in outputs:
    print("### USER ###")
    print(output.prompt)
    print("### RESPONSE ###")
    print(output.outputs[0].text)
    print("-------------------------------------------------------")

INFO 11-09 00:19:21 [__init__.py:39] Available plugins for group vllm.platform_plugins:
INFO 11-09 00:19:21 [__init__.py:41] - optimum_neuron -> optimum.neuron.vllm.plugin:register
INFO 11-09 00:19:21 [__init__.py:44] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.


/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/modules/moe/blockwise.py:42: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  component, error = import_nki(config)
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/modules/moe/blockwise

INFO 11-09 00:19:22 [__init__.py:235] Platform plugin optimum_neuron is activated
WARNING 11-09 00:19:22 [_custom_ops.py:20] Failed to import from vllm._C with ImportError('libcuda.so.1: cannot open shared object file: No such file or directory')
INFO 11-09 00:19:30 [config.py:841] This model supports multiple tasks: {'generate', 'classify', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 11-09 00:19:30 [config.py:1472] Using max model len 2048
WARNING 11-09 00:19:30 [arg_utils.py:1735] device type=neuron is not supported by the V1 Engine. Falling back to V0. 
INFO 11-09 00:19:33 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='/home/ubuntu/environment/ml/SY/qwen/compiled_model', speculative_config=None, tokenizer='/home/ubuntu/environment/ml/SY/qwen/compiled_model', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_d

/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/optimum/neuron/models/inference/backend/modules/kvcache/kv_cache_manager.py:24: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from ..attention.gqa import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/optimum/neuron/models/inference/llama/modeling_llama.py:45: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from ..backend.modules.attention.attention_base import NeuronAttentionBase
INFO:Neuron:Loading sharded checkpoint from /home/ubuntu/environment/ml/SY/qwen/compiled_model/checkpoint/weights


WARNING 11-09 00:19:42 [config.py:4861] Current vLLM config is not set.
INFO 11-09 00:19:42 [executor_base.py:113] # neuron blocks: 2, # CPU blocks: 0
INFO 11-09 00:19:42 [executor_base.py:118] Maximum concurrency for 2048 tokens per request: 2.00x
INFO 11-09 00:19:42 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 0.00 seconds


Adding requests:   0%|          | 0/2 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

2025-Nov-09 00:19:42.0213 195463:219957 [1] int nccl_net_ofi_create_plugin(nccl_net_ofi_plugin_t**):213 CCOM WARN NET/OFI Failed to initialize sendrecv protocol
2025-Nov-09 00:19:42.0223 195463:219957 [1] int nccl_net_ofi_create_plugin(nccl_net_ofi_plugin_t**):354 CCOM WARN NET/OFI aws-ofi-nccl initialization failed
2025-Nov-09 00:19:42.0233 195463:219957 [1] ncclResult_t nccl_net_ofi_init_no_atexit_fini_v6(ncclDebugLogger_t):183 CCOM WARN NET/OFI Initializing plugin failed
2025-Nov-09 00:19:42.0243 195463:219957 [1] net_plugin.cc:97 CCOM WARN OFI plugin initNet() failed is EFA enabled?
#########################################################
### USER ###

<|im_start|>system
You are a calm, medically accurate first-aid assistant. Give step-by-step help.<|im_end|>
<|im_start|>user
My friend burned his hand on a hot pan. What should I do immediately?<|im_end|>
<|im_start|>assistant

### RESPONSE ###
<think>

</think>

Stop the burning. If the wound is not bleeding, use a compress wrappe